In [1]:
# 梯度直方图 HOG  使用滑动窗口，不是基于颜色而是基于梯度


In [13]:
# 检测人
import cv2
import numpy as np
from ipdb import set_trace
def is_inside(o,i):
    ox,oy,ow,oh=o
    ix,iy,iw,ih=i
    return ox > ix and oy > iy and ox+ow < ix+iw and oy+oh < iy+ih

def drawPerson(image,person):
    x,y,w,h = person
    cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
    
img=cv2.imread(u'meinv.png')
hog=cv2.HOGDescriptor()
hog.setSVMDetector(cv2.HOGDescriptor_getDefaultPeopleDetector())
found,w = hog.detectMultiScale(img)

foundFiltered=[]
for ri ,r in enumerate(found):
    for qi ,q in enumerate(found):
        if ri != qi and is_inside(r,q):
            break
        else:
            foundFiltered.append(r)

print('foundFiltered',foundFiltered)            
for person in foundFiltered:
    drawPerson(img,person)
    
cv2.imshow('people detection',img)    
cv2.waitKey(0)
cv2.destroyAllWindows()

foundFiltered [array([327, 189, 223, 446], dtype=int32), array([327, 189, 223, 446], dtype=int32), array([327, 189, 223, 446], dtype=int32), array([  0,   0, 336, 673], dtype=int32), array([  0,   0, 336, 673], dtype=int32), array([  0,   0, 336, 673], dtype=int32), array([710, 176, 202, 403], dtype=int32), array([710, 176, 202, 403], dtype=int32), array([710, 176, 202, 403], dtype=int32)]


In [14]:
#创建好训练目标检测器
#汽车检测

In [15]:
import cv2
import numpy as np
from os.path import join

In [27]:
datapath='./CarData/CarData/TrainImages/'

def path(cls,i):
#     print( '%s/%s%d.pgm' %(datapath,cls,i+1))
    return '%s/%s%d.pgm' %(datapath,cls,i+1)

def extract_sift(fn):
    im=cv2.imread(fn,0)
#     cv2.imshow('im'+str(fn),im)
    return extract.compute(im,detect.detect(im))[1]

def bow_features(fn):
    im=cv2.imread(fn,0)
    return extract_bow.compute(im,detect.detect(im))

def predict(fn):
    f = bow_features(fn)
    p=svm.predict(f)
    print('fn \t ',p[1][0][0])
    return p

pos,neg='pos-','neg-'
detect=cv2.xfeatures2d.SIFT_create()
extract=cv2.xfeatures2d.SIFT_create()

flann_params=dict(algorithm=1,trees=5)
flann = cv2.FlannBasedMatcher(flann_params,{})

bow_kmeans_trainer = cv2.BOWKMeansTrainer(40)
extract_bow = cv2.BOWImgDescriptorExtractor(extract,flann)
for i in range(8):
    bow_kmeans_trainer.add(extract_sift(path(pos,i)))
    bow_kmeans_trainer.add(extract_sift(path(neg,i)))

voc=bow_kmeans_trainer.cluster()
extract_bow.setVocabulary(voc)

traindata,trainlabels=[],[]
for i in range(20):
    traindata.extend(bow_features(path(pos,i)))
    trainlabels.append(1)
    traindata.extend(bow_features(path(neg,i)))
    trainlabels.append(-1)
    
svm=cv2.ml.SVM_create()
svm.train(np.array(traindata),cv2.ml.ROW_SAMPLE,np.array(trainlabels))

          
car,notcar='./CarData/CarData/TestImages/test-1.pgm','./chess.png'
carImg=cv2.imread(car)
notCarImg=cv2.imread(notcar)
carPredict=predict(car)
notCarPredict=predict(notcar)

font=cv2.FONT_HERSHEY_SCRIPT_SIMPLEX

if(carPredict[1][0][0] == 1.0):
    cv2.putText(carImg,'Car detected',(10,30),font,1,(255,0,0),2,cv2.LINE_AA)
    
cv2.imshow('BOW + SVM Success',carImg)
cv2.imshow('BOW + SVM Failure',notCarImg)
cv2.waitKey(0)
cv2.destroyAllWindows()

fn 	  1.0
fn 	  -1.0


In [28]:
a=[3,4,5,6,1,8]
b=np.argsort(a)
c=b[::-1]
print(b,'\n',c)

[4 0 1 2 3 5] 
 [5 3 2 1 0 4]


In [35]:
# import the necessary packages
import numpy as np

# Malisiewicz et al.
# Python port by Adrian Rosebrock
def non_max_suppression_fast(boxes, overlapThresh):
  # if there are no boxes, return an empty list
  if len(boxes) == 0:
    return []

  # if the bounding boxes integers, convert them to floats --
  # this is important since we'll be doing a bunch of divisions
  if boxes.dtype.kind == "i":
    boxes = boxes.astype("float")

  # initialize the list of picked indexes 
    pick = []

      # grab the coordinates of the bounding boxes
    x1 = boxes[:,0]
    y1 = boxes[:,1]
    x2 = boxes[:,2]
    y2 = boxes[:,3]
    scores = boxes[:,4]
    # compute the area of the bounding boxes and sort the bounding
    # boxes by the score/probability of the bounding box
    area = (x2 - x1 + 1) * (y2 - y1 + 1)
    idxs = np.argsort(scores)[::-1]

# keep looping while some indexes still remain in the indexes
# list
    while len(idxs) > 0:
        # grab the last index in the indexes list and add the
        # index value to the list of picked indexes
        last = len(idxs) - 1
        i = idxs[last]
        pick.append(i)

    # find the largest (x, y) coordinates for the start of
    # the bounding box and the smallest (x, y) coordinates
    # for the end of the bounding box
    xx1 = np.maximum(x1[i], x1[idxs[:last]])
    yy1 = np.maximum(y1[i], y1[idxs[:last]])
    xx2 = np.minimum(x2[i], x2[idxs[:last]])
    yy2 = np.minimum(y2[i], y2[idxs[:last]])

    # compute the width and height of the bounding box
    w = np.maximum(0, xx2 - xx1 + 1)
    h = np.maximum(0, yy2 - yy1 + 1)

    # compute the ratio of overlap
    overlap = (w * h) / area[idxs[:last]]

    # delete all indexes from the index list that have
    idxs = np.delete(idxs, np.concatenate(([last],
      np.where(overlap > overlapThresh)[0])))

  # return only the bounding boxes that were picked using the
  # integer data type
    return boxes[pick].astype("int")


def sliding_window(image, step, window_size):
    for y in range(0, image.shape[0], step):
        for x in range(0, image.shape[1], step):
            yield (x, y, image[y:y + window_size[1], x:x + window_size[0]])

import cv2

def resize(img, scaleFactor):
    return cv2.resize(img, (int(img.shape[1] * (1 / scaleFactor)), int(img.shape[0] * (1 / scaleFactor))), interpolation=cv2.INTER_AREA)

def pyramid(image, scale=1.5, minSize=(200, 80)):
    yield image

    while True:
        image = resize(image, scale)
        if image.shape[0] < minSize[1] or image.shape[1] < minSize[0]:
            break

    yield image
        

In [39]:
import cv2
import numpy as np

datapath = "./CarData/CarData/TrainImages/"
SAMPLES = 400

def path(cls,i):
    return "%s/%s%d.pgm"  % (datapath,cls,i+1)

def get_flann_matcher():
    flann_params = dict(algorithm = 1, trees = 5)
    return cv2.FlannBasedMatcher(flann_params, {})

def get_bow_extractor(extract, match):
    return cv2.BOWImgDescriptorExtractor(extract, match)

def get_extract_detect():
    return cv2.xfeatures2d.SIFT_create(), cv2.xfeatures2d.SIFT_create()

def extract_sift(fn, extractor, detector):
    im = cv2.imread(fn,0)
    return extractor.compute(im, detector.detect(im))[1]
    
def bow_features(img, extractor_bow, detector):
    return extractor_bow.compute(img, detector.detect(img))

def car_detector():
    pos, neg = "pos-", "neg-"
    detect, extract = get_extract_detect()
    matcher = get_flann_matcher()
    #extract_bow = get_bow_extractor(extract, matcher)
    print ("building BOWKMeansTrainer...")
    bow_kmeans_trainer = cv2.BOWKMeansTrainer(12)
    extract_bow = cv2.BOWImgDescriptorExtractor(extract, matcher)

    print ("adding features to trainer")
    for i in range(SAMPLES):
        print (i)
        bow_kmeans_trainer.add(extract_sift(path(pos,i), extract, detect))
    #bow_kmeans_trainer.add(extract_sift(path(neg,i), extract, detect))

    vocabulary = bow_kmeans_trainer.cluster()
    extract_bow.setVocabulary(vocabulary)

    traindata, trainlabels = [],[]
    print ("adding to train data")
    for i in range(SAMPLES):
        print (i)
        traindata.extend(bow_features(cv2.imread(path(pos, i), 0), extract_bow, detect))
        trainlabels.append(1)
        traindata.extend(bow_features(cv2.imread(path(neg, i), 0), extract_bow, detect))
        trainlabels.append(-1)

    svm = cv2.ml.SVM_create()
    svm.setType(cv2.ml.SVM_C_SVC)
    svm.setGamma(1)
    svm.setC(35)
    svm.setKernel(cv2.ml.SVM_RBF)

    svm.train(np.array(traindata), cv2.ml.ROW_SAMPLE, np.array(trainlabels))
    return svm, extract_bow


In [49]:
# import urllib.request

def in_range(number, test, thresh=0.2):
    return abs(number - test) < thresh

test_image = "./CarData/CarData/TestImages_Scale/test-0.pgm"
img_path = "./CarData/CarData/TestImages_Scale/test-1.pgm"

# urllib.request.urlretrieve(test_image, img_path)

svm, extractor = car_detector()
detect = cv2.xfeatures2d.SIFT_create()

w, h = 100, 40
img = cv2.imread(img_path)
#img = cv2.imread(test_image)

rectangles = []
counter = 1
scaleFactor = 1.25
scale = 1
font = cv2.FONT_HERSHEY_PLAIN

for resized in pyramid(img, scaleFactor):  
    scale = float(img.shape[1]) / float(resized.shape[1])
    for (x, y, roi) in sliding_window(resized, 20, (100, 40)):
    
        if roi.shape[1] != w or roi.shape[0] != h:
            continue

    try:
        bf = bow_features(roi, extractor, detect)
        _, result = svm.predict(bf)
        a, res = svm.predict(bf, flags=cv2.ml.STAT_MODEL_RAW_OUTPUT | cv2.ml.STAT_MODEL_UPDATE_MODEL)
        print ("Class: %d, Score: %f, a: %s" % (result[0][0], res[0][0], res))
        score = res[0][0]
        if result[0][0] == 1:
            if score < -1.0:
                rx, ry, rx2, ry2 = int(x * scale), int(y * scale), int((x+w) * scale), int((y+h) * scale)
                rectangles.append([rx, ry, rx2, ry2, abs(score)])
    except:
        pass

    counter += 1 

windows = np.array(rectangles)
boxes = nms(windows, 0.25)


for (x, y, x2, y2, score) in boxes:
    print (x, y, x2, y2, score)
    cv2.rectangle(img, (int(x),int(y)),(int(x2), int(y2)),(0, 255, 0), 1)
    cv2.putText(img, "%f" % score, (int(x),int(y)), font, 1, (0, 255, 0))

cv2.imshow("img", img)
cv2.waitKey(0)

building BOWKMeansTrainer...
adding features to trainer
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
26

TypeError: 'NoneType' object is not iterable